In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

# %matplotlib notebook # for inttmeeractive plotting
%matplotlib inline

In [16]:
filePath = '/Users/jun/Downloads/1111.mov'

cap = cv2.VideoCapture( filePath )
i = 0
n = 20
while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret:
        if i == 0:
            prvs = frame[:, :, 2]                   # 現在のフレームを格納
            # Optical flow の着色可視化用
            hsv = np.zeros_like( frame ); hsv[...,1] = 255
            # 処理結果を保存するための動画ファイル
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            # サイズ指定で、openCV のサイズ指定はy,xであることに注意
            out = cv2.VideoWriter( '/Users/jun/Downloads/short.avi', fourcc, 30, prvs.shape[::-1] )
            
            # 速度場を描くために X,Y座標群を作る（サンプリングは表示時に行う）
            # (右辺座標規約はopenCVに従う、左辺はmatplotlib)
            Y, X = np.meshgrid( np.arange( 0, frame.shape[0] ),  # Y, X とすることに注意
                                            np.arange( 0, frame.shape[1] ) )
        else:
            next = frame[:, :, 0]                              # フレーム取得
            flow = cv2.calcOpticalFlowFarneback( # optical flow 算出 
                prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            prvs = next
            
            # Optical flow の着色可視化用
            mag, ang = cv2.cartToPolar( flow[...,0], flow[...,1] )
            hsv[..., 0] = ang * 180/np.pi/2
            hsv[..., 2] = cv2.normalize( mag, None, 0, 255, cv2.NORM_MINMAX )
            rgb = cv2.cvtColor( hsv, cv2.COLOR_HSV2BGR )
            # 速度場 (右辺座標規約はopenCVに従う、左辺はmatplotlib)
            dx = flow[...,0]
            dy = flow[...,1]
            
            fig = plt.figure( figsize = prvs.shape[::-1], dpi=1 )
            ax = fig.gca()
            ax.axis('off')
            plt.subplots_adjust(left=0, right=1, bottom=0, top=1)

            # matplotlib 座標でグラフを描画 
            plt.quiver( X[::n, ::n], Y[::n, ::n],    # ベクトル基点
                            dx[::n, ::n], -dy[::n, ::n]  #,
                            # np.sqrt( dx[::n, ::n]**2 + dy[::n, ::n]**2 ),
                            # units="xy", scale=0.0001, cmap="hot_r"
            )
            
            fig.canvas.draw() # https://stackoverflow.com/questions/51059581/matplotlib-convert-plot-to-numpy-array-without-borders
            image = np.array( fig.canvas.renderer._renderer )            
            plt.close(fig)
            # 動画としても保存しておく
            rgb = cv2.addWeighted( rgb, 0.5, frame, 0.5, 1.0)
            #rgb = cv2.addWeighted( rgb, 0.8, image[...,0:3], 0.2, 1.0)
            rgb = cv2.bitwise_xor( rgb, image[...,0:3])
            out.write( rgb )
        i +=1
    else:
        cap.release(); out.release() # 入力動画・出力動画ともに閉じる
        break